# Item Response Ranking for IRT

This notebook will show you how to train and use the IRR-IRT.
Refer to [IRR doc](../../docs/IRR.md) for more details.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a IRR-IRT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

In [7]:
import logging
from longling.lib.structure import AttrDict
from longling import set_logging_info
from EduCDM.IRR import pair_etl as etl, point_etl as vt_etl, extract_item

set_logging_info()

params = AttrDict(
    batch_size=256,
    n_neg=10,
    n_imp=10,
    logger=logging.getLogger(),
    hyper_params={"user_num": 4164}
)
item_knowledge = extract_item("../../data/a0910/item.csv", 123, params)
train_data, train_df = etl("../../data/a0910/train.csv", item_knowledge, params)
valid_data, _ = vt_etl("../../data/a0910/valid.csv", item_knowledge, params)
test_data, _ = vt_etl("../../data/a0910/test.csv", item_knowledge, params)

train_data, valid_data, test_data

rating2triplet: 100%|██████████| 17051/17051 [00:21<00:00, 805.56it/s]


(<longling.lib.iterator.AsyncLoopIter at 0x2133cafa400>,
 <torch.utils.data.dataloader.DataLoader at 0x2133c9ba2b0>)

In [8]:
train_df

,user_id,item_id,score
0,1615,12977,1.0
1,782,13124,0.0
2,1084,16475,0.0
3,593,8690,0.0
4,127,14225,1.0
...,...,...,...
186044,2280,6019,0.0
186045,121,2,1.0
186046,601,5425,1.0
186047,573,2412,0.0


In [9]:
from EduCDM.IRR import IRT

cdm = IRT(
    4163 + 1,
    17746 + 1,
    123
)
cdm.train(
    train_data,
    valid_data,
    epoch=2,
)
cdm.save("IRR-IRT.params")

Epoch 0: 727it [02:02,  5.92it/s]
formatting item df: 100%|██████████| 10415/10415 [00:01<00:00, 9749.68it/s]
ranking metrics: 10415it [00:40, 254.13it/s]
formatting item df: 100%|██████████| 10415/10415 [00:01<00:00, 9593.92it/s]
ranking metrics: 10415it [00:42, 246.39it/s]
INFO:root:save parameters to IRR-IRT.params


[Epoch 0] Loss: 6.042366, PointLoss: 0.676846, PairLoss: 11.407887
[Epoch 0]
      ndcg@k  precision@k  recall@k      f1@k     len@k  support@k  ndcg@k(B)  \
1   1.000000     0.674988  0.475920  0.527342  1.000000      10415   1.000000   
3   0.891091     0.674540  0.737472  0.685684  1.906961      10415   0.416033   
5   0.893729     0.674165  0.794214  0.711436  2.229573      10415   0.424743   
10  0.893938     0.673999  0.816025  0.720019  2.423428      10415   0.425957   

    precision@k(B)  recall@k(B)   f1@k(B)  len@k(B)  support@k(B)  
1         0.326452     0.253676  0.273340       1.0         10415  
3         0.211522     0.434936  0.268588       3.0         10415  
5         0.149304     0.481618  0.215243       5.0         10415  
10        0.082180     0.502223  0.134561      10.0         10415  
auc: 0.839379	map: 0.912738	mrr: 0.904580	coverage_error: 3.005876	ranking_loss: 0.281039	len: 2.458569	support: 10415	map(B): 0.888846	mrr(B): 0.902716
[Epoch 1] Loss: 6.027494

In [10]:
from EduCDM.IRR.metrics import result_format
cdm.load("IRR-IRT.params")
print(result_format(cdm.eval(test_data)))

INFO:root:load parameters from IRR-IRT.params
formatting item df: 100%|██████████| 13682/13682 [00:01<00:00, 8991.02it/s]
ranking metrics: 13682it [01:00, 227.23it/s]


      ndcg@k  precision@k  recall@k      f1@k     len@k  support@k  ndcg@k(B)  \
1   1.000000     0.668981  0.370194  0.434254  1.000000      13682   1.000000   
3   0.861122     0.668993  0.663701  0.634487  2.268528      13682   0.447512   
5   0.868104     0.668187  0.770498  0.690224  2.981582      13682   0.467525   
10  0.869298     0.667871  0.844868  0.723768  3.723652      13682   0.472620   

    precision@k(B)  recall@k(B)   f1@k(B)  len@k(B)  support@k(B)  
1         0.333065     0.215427  0.243171       1.0         13682  
3         0.250597     0.420785  0.286807       3.0         13682  
5         0.200746     0.511538  0.263657       5.0         13682  
10        0.127386     0.579823  0.192924      10.0         13682  
auc: 0.768879	map: 0.869789	mrr: 0.872195	coverage_error: 4.651518	ranking_loss: 0.318189	len: 4.075428	support: 13682	map(B): 0.818462	mrr(B): 0.871107
